<a href="https://colab.research.google.com/github/SamsonOluwaseun/All_About_Analytics/blob/main/Customer_MDM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Define the current date as a reference for CreationDate simulation
current_date = datetime.now().date()
np.random.seed(42) # for reproducibility

# --- Helper function to generate simulated records (Modified for ID format) ---
def generate_customer_records_complex(num_records, source_id_type):
    """Generates complex simulated customer records, tailoring the ID format to the source."""
    records = []

    # Base list of common customers (to ensure deliberate matches and duplicates) - Unchanged PII Data
    base_customers = [
        ('John', 'Smith', '1985-05-15', '123 Main St, Anytown', 'M', '555-1001'),
        ('Jon', 'Smith', '1985-05-15', '123 Main Street', 'Male', '5551001'),
        ('J. J.', 'Smyth', '1985-05-15', '123 Main St Apt 2B', '1', '5551001'),
        ('Jane', 'Doe', '1990-11-20', '45 Oak Ave, City', 'F', '555-2002'),
        ('Jaine', 'Doh', '1990-11-20', '45 Oak Avenue', 'Female', '5552002'),
        ('Peter', 'Jones', '1975-02-01', '789 Pine Ln', 'MALE', '555-3003'),
        ('P.', 'Jones', '1975-02-01', '789 Pine Lane', 'M', '555-3003'),
        ('Mary', 'Williams', '1988-08-08', '33 River Rd', 'F', '555-4004'),
        ('Alice', 'Brown', '1995-01-01', '50 High St', 'F', '555-5005'),
        ('Robert', 'White', '1970-12-25', '10 Elm Dr', 'M', '555-6006'),
        ('Chris', 'Green', '2000-03-10', '90 Park Ave', 'M', '555-7007'),
    ]

    # Pad the base list to ensure we reach num_records
    while len(base_customers) < num_records:
        base_customers.append((
            np.random.choice(['David', 'Laura', 'Mark', 'Sarah']),
            np.random.choice(['Taylor', 'Clark', 'Hall', 'Baker']),
            (current_date - timedelta(days=np.random.randint(7000, 20000))).strftime('%Y-%m-%d'),
            f'{np.random.randint(10, 999)} Random Road',
            np.random.choice(['M', 'F']),
            f'555-{np.random.randint(1000, 9999)}'
        ))


    records_list = []
    for i in range(num_records):
        base_idx = i % len(base_customers)
        first_name, last_name, dob, address, sex, phone = base_customers[base_idx]
        creation_date = (current_date - timedelta(days=np.random.randint(30, 1000))).strftime('%Y-%m-%d')

        # --- ID FORMAT LOGIC ---
        if source_id_type == 'S1_Policy_ID':
            # 1. Full Integer (e.g., 100100, 100101, ...)
            source_id = 100000 + i
        elif source_id_type == 'S2_Claimant_ID':
            # 2. Alpha-Numeric 5-digit (e.g., CL-001, CL-002, ...)
            source_id = f'CL-{i+1:03d}'
        elif source_id_type == 'S3_CRM_ID':
            # 3. Alpha-Numeric 8-digit (e.g., CUST-1000, CUST-1001, ...)
            source_id = f'CUST-{1000 + i}'
        else:
             source_id = i + 1000 # Fallback
        # -----------------------

        record = {
            f'{source_id_type}': source_id,
            'First_Name': first_name,
            'Last_Name': last_name,
            'DOB': dob,
            'Sex': sex,
            'Address_Line': address,
            'Creation_Timestamp': creation_date,
            'Phone_Number': phone
        }
        records_list.append(record)

    return pd.DataFrame(records_list)


In [2]:
# ====================================================================================
# --- 2. Create Table According to Sources (20 Records Each with Varied IDs) ---
# ====================================================================================

# S1: Life Insurance Policy Admin (Legacy) - Full Integer ID
df_s1_raw = generate_customer_records_complex(20, 'S1_Policy_ID').rename(columns={
    'First_Name': 'Cust_FName', 'Last_Name': 'Cust_LName', 'DOB': 'Birth_Date',
    'Sex': 'Gender_Code', 'Address_Line': 'Policy_Address',
    'Creation_Timestamp': 'S1_Creation_DTS', 'Phone_Number': 'Phone'
})
df_s1_raw['SourceSystem'] = 'S1_Life'
df_s1_raw['S1_Policy_ID'] = df_s1_raw['S1_Policy_ID'].astype(int) # Ensure S1 ID is integer

# S2: Auto Insurance Claims System - 5-digit Alpha-Numeric ID
df_s2_raw = generate_customer_records_complex(20, 'S2_Claimant_ID').rename(columns={
    'First_Name': 'FName', 'Last_Name': 'LName', 'DOB': 'DOB',
    'Sex': 'Sex', 'Address_Line': 'Claimant_Addr',
    'Creation_Timestamp': 'Rec_Create_Dt', 'Phone_Number': 'Claim_Phone'
})
# S2 internal duplicate to test matching robustness
df_s2_raw.loc[0, 'FName'] = 'John'
df_s2_raw.loc[1, 'FName'] = 'John'
df_s2_raw['SourceSystem'] = 'S2_Claims'


# S3: CRM System (Modern) - 8-digit Alpha-Numeric ID
df_s3_raw = generate_customer_records_complex(20, 'S3_CRM_ID').rename(columns={
    'First_Name': 'Customer_First', 'Last_Name': 'Customer_Last', 'DOB': 'Date_Of_Birth',
    'Sex': 'Sex', 'Address_Line': 'Primary_Address',
    'Creation_Timestamp': 'CRM_Record_Date', 'Phone_Number': 'Contact_Phone'
})
# S3 data quality formatting
df_s3_raw.loc[df_s3_raw['Sex'] == 'M', 'Sex'] = 'Male'
df_s3_raw.loc[df_s3_raw['Sex'] == 'F', 'Sex'] = 'Female'
df_s3_raw['SourceSystem'] = 'S3_CRM'

print("--- Source DataFrames with Distinct ID Formats ---")
print(f"S1_Life ID Example: {df_s1_raw['S1_Policy_ID'].iloc[0]} (Type: {df_s1_raw['S1_Policy_ID'].dtype})")
print(f"S2_Claims ID Example: {df_s2_raw['S2_Claimant_ID'].iloc[0]} (Type: {df_s2_raw['S2_Claimant_ID'].dtype})")
print(f"S3_CRM ID Example: {df_s3_raw['S3_CRM_ID'].iloc[0]} (Type: {df_s3_raw['S3_CRM_ID'].dtype})")
print("-" * 70)


# CONSOLIDATION: Combine all source tables into the Staging Area
raw_df = pd.concat([df_s1_raw, df_s2_raw, df_s3_raw], ignore_index=True)
raw_df.index.name = 'Raw_Record_ID'

print(f"\n--- Consolidated Raw Data (Staging Area) - Total {len(raw_df)} Records ---")
# Display the different ID formats
print(raw_df[['SourceSystem', 'S1_Policy_ID', 'S2_Claimant_ID', 'S3_CRM_ID',
              'Cust_FName', 'FName', 'Customer_First', 'S1_Creation_DTS']].head(10).fillna('-'))
print("\n" + "="*80 + "\n")


--- Source DataFrames with Distinct ID Formats ---
S1_Life ID Example: 100000 (Type: int64)
S2_Claims ID Example: CL-001 (Type: object)
S3_CRM ID Example: CUST-1000 (Type: object)
----------------------------------------------------------------------

--- Consolidated Raw Data (Staging Area) - Total 60 Records ---
              SourceSystem  S1_Policy_ID S2_Claimant_ID S3_CRM_ID Cust_FName  \
Raw_Record_ID                                                                  
0                  S1_Life      100000.0              -         -       John   
1                  S1_Life      100001.0              -         -        Jon   
2                  S1_Life      100002.0              -         -      J. J.   
3                  S1_Life      100003.0              -         -       Jane   
4                  S1_Life      100004.0              -         -      Jaine   
5                  S1_Life      100005.0              -         -      Peter   
6                  S1_Life      100006.0    

In [6]:
# ====================================================================================
# --- 3. Table Transformation and Consolidation Script (Rule-Based Standardization) ---
# ====================================================================================

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Define the canonical column names for the MDM Hub
CANONICAL_COLUMNS = {
    'First_Name': ['Cust_FName', 'FName', 'Customer_First'],
    'Last_Name': ['Cust_LName', 'LName', 'Customer_Last'],
    'DOB': ['Birth_Date', 'DOB', 'Date_Of_Birth'],
    'Sex': ['Gender_Code', 'Sex'],
    'Address': ['Policy_Address', 'Claimant_Addr', 'Primary_Address'],
    'Phone': ['Phone', 'Claim_Phone', 'Contact_Phone'],
    'CreationDate': ['S1_Creation_DTS', 'Rec_Create_Dt', 'CRM_Record_Date']
}

def standardize_data(df, canonical_map):
    """
    Performs data mapping, standardization, and cleans data into a canonical model.
    Uses basic cleansing combined with recency-based rules for First/Last Name.
    """
    df_clean = df.copy()

    # 1. MAP DISPARATE COLUMNS TO CANONICAL NAMES
    for canonical, source_list in canonical_map.items():
        df_clean[canonical] = df_clean.apply(
            lambda row: next((row[col] for col in source_list if col in row and pd.notna(row[col])), None),
            axis=1
        )

    # Ensure Dates are ready for sorting
    df_clean['DOB'] = pd.to_datetime(df_clean['DOB'], errors='coerce').dt.date
    df_clean['CreationDate'] = pd.to_datetime(df_clean['CreationDate'], errors='coerce')

    # 2. STANDARDIZATION AND CLEANSING (Canonical Fields)

    # A. Initial Cleanse (Standardize format before applying rules)
    # This addresses obvious, non-semantic variations like casing or initials.
    df_clean['FirstName_C'] = df_clean['First_Name'].astype(str).str.strip().str.upper().str.replace('.', '', regex=False)
    df_clean['LastName_C'] = df_clean['Last_Name'].astype(str).str.strip().str.upper()

    # B. Rule-Based Name Standardization (Recency Rule)
    # We will prioritize the longest, most recent name for groups of potential matches.
    # Since we don't have the final match groups yet (that's Step 4), we apply the rule
    # *conditionally* on records that share key PII (DOB, Last Name) that is typically stable.

    # 1. Group records that are highly likely to be the same person
    group_cols = ['LastName_C', 'DOB']

    # 2. Within each group, determine the "best" FirstName and LastName using a complex rule:
    #    Rule: MAX(Recency) then MAX(Length)
    def determine_best_name(group, name_col):
        # Sort by CreationDate (Most Recent first), then by name length (Longest first)
        group_sorted = group.sort_values(by=['CreationDate', name_col], ascending=[False, False])

        # Return the value from the top record
        return group_sorted[name_col].iloc[0]

    # Apply the rule to create the final standardized columns (_S)
    df_clean['FirstName_S'] = df_clean.groupby(group_cols, dropna=False)['FirstName_C'].transform(
        lambda x: determine_best_name(df_clean.loc[x.index], 'FirstName_C')
    )

    df_clean['LastName_S'] = df_clean.groupby(group_cols, dropna=False)['LastName_C'].transform(
        lambda x: determine_best_name(df_clean.loc[x.index], 'LastName_C')
    )

    # C. Other Standardizations (As before)
    df_clean['Sex_S'] = df_clean['Sex'].astype(str).str.upper().replace({'M': 'MALE', '1': 'MALE', 'F': 'FEMALE', '2': 'FEMALE'}, regex=False)

    df_clean['Phone_S'] = df_clean['Phone'].astype(str).str.replace(r'[^0-9]', '', regex=True)
    df_clean['Phone_S'] = df_clean.apply(
        lambda row: f'{row["Phone_S"][:3]}-{row["Phone_S"][3:6]}-{row["Phone_S"][6:10]}'
        if len(row["Phone_S"]) == 10 else None, axis=1
    )
    df_clean['Address_S'] = df_clean['Address'].astype(str).str.upper().str.replace(' AVENUE', ' AVE', regex=False).str.replace(' ROAD', ' RD', regex=False).str.strip()

    # 3. SELECT FINAL STAGING COLUMNS
    cols_to_keep = ['Raw_Record_ID', 'SourceSystem', 'S1_Policy_ID', 'S2_Claimant_ID', 'S3_CRM_ID', 'CreationDate']
    cols_to_keep.extend(['FirstName_S', 'LastName_S', 'DOB', 'Sex_S', 'Address_S', 'Phone_S'])

    return df_clean[cols_to_keep]

# --- Note: You must run the data generation (Step 2) before running this ---

# Assuming 'raw_df' exists from Step 2 and needs the index reset fix:
# The Raw_Record_ID is currently the DataFrame index. Resetting it converts it into a regular column.
raw_df = raw_df.reset_index() # Uncomment this line if running in sequence

staged_df = standardize_data(raw_df, CANONICAL_COLUMNS)

# print("--- Staged Data (Canonical Model Ready for Matching) - First 8 Records ---")
print(staged_df[['SourceSystem', 'S1_Policy_ID', 'S2_Claimant_ID', 'S3_CRM_ID', 'FirstName_S', 'LastName_S', 'CreationDate']].head(8).fillna('-'))

  SourceSystem  S1_Policy_ID S2_Claimant_ID S3_CRM_ID FirstName_S LastName_S  \
0      S1_Life      100000.0              -         -         JON      SMITH   
1      S1_Life      100001.0              -         -         JON      SMITH   
2      S1_Life      100002.0              -         -         J J      SMYTH   
3      S1_Life      100003.0              -         -        JANE        DOE   
4      S1_Life      100004.0              -         -       JAINE        DOH   
5      S1_Life      100005.0              -         -       PETER      JONES   
6      S1_Life      100006.0              -         -       PETER      JONES   
7      S1_Life      100007.0              -         -        MARY   WILLIAMS   

  CreationDate  
0   2023-12-18  
1   2025-09-04  
2   2023-06-07  
3   2025-04-11  
4   2024-12-25  
5   2024-09-02  
6   2024-02-02  
7   2024-11-13  


In [7]:
# --- 4. Python Script for Matching Rules to Create a Master Record with Master ID ---

def perform_matching_and_mastering(staged_df):
    """Applies matching rules to link records and create a Master ID."""

    # Initialize Master_ID column
    staged_df['Master_ID'] = None
    next_master_id = 1

    # Matching Logic: Prioritized Rules

    # 1. EXACT Match Rule: Full Name + DOB + Phone
    match_cols_exact = ['FirstName_S', 'LastName_S', 'DOB', 'Phone_S']
    exact_matches = staged_df.dropna(subset=match_cols_exact).groupby(match_cols_exact)

    for _, group in exact_matches:
        if len(group) > 1:
            # If a match is found, use the lowest existing Master_ID or create a new one
            existing_mid = group['Master_ID'].dropna().min()
            new_mid = existing_mid if pd.notna(existing_mid) else f'CMD_{next_master_id:03d}'
            staged_df.loc[group.index, 'Master_ID'] = new_mid
            if pd.isna(existing_mid):
                next_master_id += 1

    # 2. FUZZY Match Rule (Secondary): Full Name + DOB (Ignore Phone/Address variations)
    match_cols_fuzzy = ['FirstName_S', 'LastName_S', 'DOB']
    fuzzy_matches = staged_df.dropna(subset=match_cols_fuzzy).groupby(match_cols_fuzzy)

    for _, group in fuzzy_matches:
        if len(group) > 1:
            # If a match is found and not yet assigned a Master_ID
            unassigned_indices = group[group['Master_ID'].isna()].index

            # Use the lowest existing Master_ID from the group, or create a new one
            existing_mid = group['Master_ID'].dropna().min()
            new_mid = existing_mid if pd.notna(existing_mid) else f'CMD_{next_master_id:03d}'

            staged_df.loc[group.index, 'Master_ID'] = new_mid
            if pd.isna(existing_mid) and len(unassigned_indices) > 0:
                next_master_id += 1

    # 3. Handle Remaining Records (Singletons)
    unassigned_indices = staged_df[staged_df['Master_ID'].isna()].index
    for idx in unassigned_indices:
        staged_df.loc[idx, 'Master_ID'] = f'CMD_{next_master_id:03d}'
        next_master_id += 1

    # --- Survivorship (For Registry Style - Consolidate IDs) ---
    # Group all records by the final Master_ID to consolidate source IDs
    master_records = staged_df.groupby('Master_ID').agg(
        S1_ID=('S1_Policy_ID', lambda x: x.dropna().unique().tolist()),
        S2_ID=('S2_Claimant_ID', lambda x: x.dropna().unique().tolist()),
        S3_ID=('S3_CRM_ID', lambda x: x.dropna().unique().tolist()),
    ).reset_index()

    # Simplify the lists to just the first ID for demo, or keep as list
    master_records['S1_ID'] = master_records['S1_ID'].apply(lambda x: x[0] if x else None)
    master_records['S2_ID'] = master_records['S2_ID'].apply(lambda x: x[0] if x else None)
    master_records['S3_ID'] = master_records['S3_ID'].apply(lambda x: x[0] if x else None)

    # For a Registry Style, the final table only needs the mapping
    mapping_df = staged_df[['Master_ID', 'S1_Policy_ID', 'S2_Claimant_ID', 'S3_CRM_ID']].copy()

    return mapping_df.drop_duplicates(subset=['Master_ID']), master_records

mapping_df, master_records = perform_matching_and_mastering(staged_df)

In [14]:
print(staged_df.head(10).fillna('-') )

   Raw_Record_ID SourceSystem  S1_Policy_ID S2_Claimant_ID S3_CRM_ID  \
0              0      S1_Life      100000.0              -         -   
1              1      S1_Life      100001.0              -         -   
2              2      S1_Life      100002.0              -         -   
3              3      S1_Life      100003.0              -         -   
4              4      S1_Life      100004.0              -         -   
5              5      S1_Life      100005.0              -         -   
6              6      S1_Life      100006.0              -         -   
7              7      S1_Life      100007.0              -         -   
8              8      S1_Life      100008.0              -         -   
9              9      S1_Life      100009.0              -         -   

  CreationDate FirstName_S LastName_S         DOB   Sex_S  \
0   2023-12-18         JON      SMITH  1985-05-15    MALE   
1   2025-09-04         JON      SMITH  1985-05-15    MALE   
2   2023-06-07         J

In [13]:
print(mapping_df.head(10).fillna('-') )

   Master_ID  S1_Policy_ID S2_Claimant_ID S3_CRM_ID
0    CMD_006      100000.0              -         -
2    CMD_003      100002.0              -         -
3    CMD_005      100003.0              -         -
4    CMD_004      100004.0              -         -
5    CMD_008      100005.0              -         -
7    CMD_007      100007.0              -         -
8    CMD_001      100008.0              -         -
9    CMD_009      100009.0              -         -
10   CMD_002      100010.0              -         -
11   CMD_010      100011.0              -         -


In [12]:
print(master_records.head(10).fillna('-') )

  Master_ID     S1_ID   S2_ID      S3_ID
0   CMD_001  100008.0  CL-009  CUST-1008
1   CMD_002  100010.0  CL-011  CUST-1010
2   CMD_003  100002.0  CL-003  CUST-1002
3   CMD_004  100004.0  CL-005  CUST-1004
4   CMD_005  100003.0  CL-004  CUST-1003
5   CMD_006  100000.0  CL-001  CUST-1000
6   CMD_007  100007.0  CL-008  CUST-1007
7   CMD_008  100005.0  CL-006  CUST-1005
8   CMD_009  100009.0  CL-010  CUST-1009
9   CMD_010  100011.0       -          -


In [17]:
# ====================================================================================
# --- 5. Survivorship and Golden Record Creation Script (Recency Rule) ---
# ====================================================================================

# Note: This script assumes 'staged_df' has been updated with the 'Master_ID' column
# from the execution of the Step 4 script.

# Define the canonical source columns whose values will be used for survivorship
# Corrected to use standardized column names from staged_df
SOURCE_VALUE_COLS = ['FirstName_S', 'LastName_S', 'DOB', 'Sex_S', 'Address_S', 'Phone_S']

# Helper function for Recency Survivorship (Accesses the canonical source value)
def get_most_recent_value(series, full_df):
    """
    Retrieves the attribute value corresponding to the maximum CreationDate
    for the group of records.
    """
    # 1. Get the indices of the records in the current group
    group_indices = series.index

    # 2. Access the CreationDate column for these specific records
    creation_dates = full_df.loc[group_indices]['CreationDate']

    # 3. Find the index corresponding to the latest non-missing CreationDate
    if creation_dates.dropna().empty:
        # Fallback: if all dates are NaT, just return the first non-NaN value
        return series.iloc[0] if not series.dropna().empty else None

    best_record_index = creation_dates.dropna().idxmax()

    # 4. Return the attribute value from the winning record
    # Access the value from the correct standardized column
    return full_df.loc[best_record_index][series.name]


# 1. Aggregate to the Master_ID level using the robust .agg() method
# This step creates the Golden Record (Master_* attributes) and the full X-Ref
master_data = staged_df.groupby('Master_ID', dropna=False).agg(

    # X-Ref Aggregation: Concatenate all linked source IDs
    S1_ID=('S1_Policy_ID', lambda x: ', '.join(x.dropna().astype(str).unique())),
    S2_ID=('S2_Claimant_ID', lambda x: ', '.join(x.dropna().astype(str).unique())),
    S3_ID=('S3_CRM_ID', lambda x: ', '.join(x.dropna().astype(str).unique())),

    # Golden Record Attributes: Apply Recency Survivorship on standardized attributes
    Master_FirstName=('FirstName_S', lambda x: get_most_recent_value(x, staged_df)),
    Master_LastName=('LastName_S', lambda x: get_most_recent_value(x, staged_df)),
    Master_DOB=('DOB', lambda x: get_most_recent_value(x, staged_df)),
    Master_Sex=('Sex_S', lambda x: get_most_recent_value(x, staged_df)),
    Master_Address=('Address_S', lambda x: get_most_recent_value(x, staged_df)),
    Master_Phone=('Phone_S', lambda x: get_most_recent_value(x, staged_df)),
).reset_index()


# 2. Final Clean-up and Display Formatting
final_result_df = master_data.copy()

# Replace empty ID strings with '-' for clean output
final_result_df['S1_ID'] = final_result_df['S1_ID'].replace('', '-')
final_result_df['S2_ID'] = final_result_df['S2_ID'].replace('', '-')
final_result_df['S3_ID'] = final_result_df['S3_ID'].replace('', '-')

print("\n" + "="*80)
print("--- Step 5: FINAL GOLDEN RECORD & CROSS-REFERENCE TABLE (X-Ref) ---")
print("Golden Record attributes determined by **Most Recent CreationDate** rule.")
print("="*80)

# Select and rename columns for the comprehensive final output
final_display_cols = [
    'Master_ID',
    'Master_FirstName', 'Master_LastName', 'Master_DOB', 'Master_Sex',
    'Master_Address', 'Master_Phone',
    'S1_ID', 'S2_ID', 'S3_ID'
]

print(final_result_df[final_display_cols].rename(columns={
    'S1_ID': 'Source 1 ID (Life)',
    'S2_ID': 'Source 2 ID (Claims)',
    'S3_ID': 'Source 3 ID (CRM)'
}))


--- Step 5: FINAL GOLDEN RECORD & CROSS-REFERENCE TABLE (X-Ref) ---
Golden Record attributes determined by **Most Recent CreationDate** rule.
   Master_ID Master_FirstName Master_LastName  Master_DOB Master_Sex  \
0    CMD_001            ALICE           BROWN  1995-01-01     FEMALE   
1    CMD_002            CHRIS           GREEN  2000-03-10       MALE   
2    CMD_003              J J           SMYTH  1985-05-15       MALE   
3    CMD_004            JAINE             DOH  1990-11-20     FEMALE   
4    CMD_005             JANE             DOE  1990-11-20     FEMALE   
5    CMD_006              JON           SMITH  1985-05-15       MALE   
6    CMD_007             MARY        WILLIAMS  1988-08-08     FEMALE   
7    CMD_008            PETER           JONES  1975-02-01       MALE   
8    CMD_009           ROBERT           WHITE  1970-12-25       MALE   
9    CMD_010             MARK           BAKER  2004-04-17       MALE   
10   CMD_011            DAVID          TAYLOR  1990-12-13       M

In [45]:
## 6. ID Lookup Script (Simulating Data Syndication)

import pandas as pd # Ensure pandas is imported if this is run independently

def lookup_source_ids_by_master_id(master_id: str, x_ref_df: pd.DataFrame):
    """
    Simulates an API call to the MDM Hub to retrieve all linked source IDs
     and the Golden Record attributes for a given Master ID, printing the IDs
     as part of the main output block.

    Returns a dictionary including the Master ID and all found source IDs.
    """
    if master_id not in x_ref_df['Master_ID'].values:
        print(f"\n❌ ERROR: Master ID '{master_id}' not found.")
        return None

    # Get the row corresponding to the Master ID
    result = x_ref_df[x_ref_df['Master_ID'] == master_id].iloc[0]

    # --- Prepare Linked Source IDs ---
    available_ids = {
        'Source 1 ID (Life)': result.get('S1_ID', '-'),
        'Source 2 ID (Claims)': result.get('S2_ID', '-'),
        'Source 3 ID (CRM)': result.get('S3_ID', '-')
    }
    found_ids = {}

    # Collect found IDs and format them for printing/return
    for source, source_id in available_ids.items():
        if source_id != '-' and pd.notna(source_id) and source_id != '':
            found_ids[source] = source_id

    # --- Print Consolidated Lookup Result ---
    print(f"\n--- Lookup Result for Master ID: {master_id} ---")

    # Golden Record Attributes (Consolidated Print)
    print("\n✅ Golden Record Attributes & X-Reference:")

    # PII/Golden Attributes
    print(f"  - First Name: {result.get('Master_FirstName', '-')}")
    print(f"  - Last Name: {result.get('Master_LastName', '-')}")
    print(f"  - DOB: {result.get('Master_DOB', '-')}")
    print(f"  - Address: {result.get('Master_Address', '-')}")
    print(f"  - Phone: {result.get('Master_Phone', '-')}")

    # Source IDs (Printed immediately after PII)
    for source, source_id in found_ids.items():
        print(f"  - {source}: {source_id}")

    # --- Return the Master ID and the found source IDs (No Change to Return) ---
    return {
        'Master_ID': master_id,
        'Linked_Source_IDs': found_ids
    }

In [47]:
# --- Example Lookup 1: A customer with records in multiple sources (John Smith Group) ---
# Master ID 100 links John/Jon/J.J. Smith records from S1, S2, and S3
lookup_source_ids_by_master_id('CMD_001', final_result_df)

# --- Example Lookup 2: A customer with a record in only one source (Unique record) ---
# Robert White is usually assigned Master ID 107 (check your output if running the full script)
# We'll pick an ID that only links to one system, e.g., Alice Brown (usually MID-103)
lookup_source_ids_by_master_id('CMD_003', final_result_df)

# --- Example Lookup 3: A non-existent ID ---
lookup_source_ids_by_master_id('CMD_109', final_result_df)

# --- Example Lookup 4: A non-existent ID ---
lookup_source_ids_by_master_id('CMD_010', final_result_df)


--- Lookup Result for Master ID: CMD_001 ---

✅ Golden Record Attributes & X-Reference:
  - First Name: ALICE
  - Last Name: BROWN
  - DOB: 1995-01-01
  - Address: 50 HIGH ST
  - Phone: None
  - Source 1 ID (Life): 100008.0
  - Source 2 ID (Claims): CL-009
  - Source 3 ID (CRM): CUST-1008

--- Lookup Result for Master ID: CMD_003 ---

✅ Golden Record Attributes & X-Reference:
  - First Name: J J
  - Last Name: SMYTH
  - DOB: 1985-05-15
  - Address: 123 MAIN ST APT 2B
  - Phone: None
  - Source 1 ID (Life): 100002.0
  - Source 2 ID (Claims): CL-003
  - Source 3 ID (CRM): CUST-1002

❌ ERROR: Master ID 'CMD_109' not found.

--- Lookup Result for Master ID: CMD_010 ---

✅ Golden Record Attributes & X-Reference:
  - First Name: MARK
  - Last Name: BAKER
  - DOB: 2004-04-17
  - Address: 280 RANDOM RD
  - Phone: None
  - Source 1 ID (Life): 100011.0


{'Master_ID': 'CMD_010',
 'Linked_Source_IDs': {'Source 1 ID (Life)': '100011.0'}}

In [44]:
print(final_result_df.head(10).fillna('-') )

  Master_ID               S1_ID           S2_ID                 S3_ID  \
0   CMD_001            100008.0          CL-009             CUST-1008   
1   CMD_002            100010.0          CL-011             CUST-1010   
2   CMD_003            100002.0          CL-003             CUST-1002   
3   CMD_004            100004.0          CL-005             CUST-1004   
4   CMD_005            100003.0          CL-004             CUST-1003   
5   CMD_006  100000.0, 100001.0  CL-001, CL-002  CUST-1000, CUST-1001   
6   CMD_007            100007.0          CL-008             CUST-1007   
7   CMD_008  100005.0, 100006.0  CL-006, CL-007  CUST-1005, CUST-1006   
8   CMD_009            100009.0          CL-010             CUST-1009   
9   CMD_010            100011.0               -                     -   

  Master_FirstName Master_LastName  Master_DOB Master_Sex      Master_Address  \
0            ALICE           BROWN  1995-01-01     FEMALE          50 HIGH ST   
1            CHRIS           GREEN